# Models and Evaluation

In [ ]:
import pandas as pd
import numpy as np

complete_data = pd.read_csv('data_processed/data.csv')
data = complete_data.copy()

data.drop('status', inplace=True, axis=1)
data = data.values
target = complete_data['status'].values
print(data)
print(target)

In [ ]:
from sklearn.model_selection import train_test_split

(training_inputs,
 testing_inputs,
 training_classes,
 testing_classes) = train_test_split(data, target, test_size=0.25, random_state=1)

In [ ]:
def recall(cm, i):
    return cm[i][i]/sum(cm[i])

def precision(cm, i):
    cmt = np.copy(cm).transpose()
    return cmt[i][i]/sum(cmt[i])

def f_measure(cm, i):
    p = precision(cm, i)*100
    r = recall(cm, i)*100
    return 2 * (p * r) / (p + r)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
def confusion_matrix(target_test, target_predictions):    
    return confusion_matrix(target_test, target_predictions)

def display_confusion_matrix(cm):
    ConfusionMatrixDisplay(cm, display_labels=target).plot()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(training_inputs, training_classes)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(training_inputs, training_classes)

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(training_inputs, training_classes)

In [ ]:
print("DTC:", dtc.score(testing_inputs, testing_classes))
print("RF :", rf.score(testing_inputs, testing_classes))
print("SVC:", svc.score(testing_inputs, testing_classes))

In [ ]:
dtc_pred = dtc.predict(testing_inputs)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
def conf_matrix(testing_classes, predicted_classes):
    cm =  confusion_matrix(testing_classes, predicted_classes)
    print("TP:", cm[1][1])
    print("TN:", cm[0][0])
    print("FP:", cm[0][1])
    print("FN:", cm[1][0])

    ConfusionMatrixDisplay(cm).plot()